In [1]:
import pandas as pd

df = pd.read_csv("Bainbridge_Departures_2013_with_Seconds_Late.csv", parse_dates = ["timestamps"], index_col = "timestamps")

In [2]:
df.head()

,departure_terminal,arrival_terminal,scheduled_departure,actual_departure,Actual_Arrival,TransitTime,seconds_late
timestamps,,,,,,,
2013-01-01 00:55:00,Bainbridge,Colman,55:00.0,05:56.0,37:37.0,32.0,656.0
2013-01-01 04:45:00,Bainbridge,NaN,45:00.0,NaN,NaN,NaN,NaN
2013-01-01 05:20:00,Bainbridge,Colman,20:00.0,19:01.0,49:40.0,30.0,-59.0
2013-01-01 06:20:00,Bainbridge,NaN,20:00.0,NaN,NaN,NaN,NaN
2013-01-01 07:05:00,Bainbridge,NaN,05:00.0,06:16.0,NaN,NaN,76.0


This time series data is not consistent. The goal of the notebook is to use Linear Interpolation to generate consiste timestamps for every 5 minutes.

In [3]:
df.tail()

,departure_terminal,arrival_terminal,scheduled_departure,actual_departure,Actual_Arrival,TransitTime,seconds_late
timestamps,,,,,,,
2013-12-31 19:10:00,Bainbridge,Colman,10:00.0,10:03.0,44:26.0,34.0,3.0
2013-12-31 20:10:00,Bainbridge,Colman,10:00.0,10:53.0,41:57.0,31.0,53.0
2013-12-31 20:55:00,Bainbridge,Colman,55:00.0,55:00.0,26:36.0,31.0,0.0
2013-12-31 21:45:00,Bainbridge,Colman,45:00.0,45:17.0,17:09.0,32.0,17.0
2013-12-31 22:35:00,Bainbridge,Colman,35:00.0,38:45.0,13:36.0,35.0,225.0


In [4]:
import datetime

d = datetime.datetime(2013, 1, 1, 1,0,0)

In [5]:
example = df.iloc[0]
example.scheduled_departure = float("nan")
example.actual_departure = float("nan")
example.Actual_Arrival = float("nan")
example.TransitTime = float("nan")
example.seconds_late = float("nan")

C:\Users\Keith\Anaconda3\lib\site-packages\pandas\core\generic.py:5096: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [6]:
example

departure_terminal     Bainbridge
arrival_terminal           Colman
scheduled_departure           NaN
actual_departure              NaN
Actual_Arrival                NaN
TransitTime                   NaN
seconds_late                  NaN
Name: 2013-01-01 00:55:00, dtype: object

In [7]:
example.name = d

In [8]:
example

departure_terminal     Bainbridge
arrival_terminal           Colman
scheduled_departure           NaN
actual_departure              NaN
Actual_Arrival                NaN
TransitTime                   NaN
seconds_late                  NaN
Name: 2013-01-01 01:00:00, dtype: object

In [9]:
df.append(example).sort_values(by = "timestamps").head()

,departure_terminal,arrival_terminal,scheduled_departure,actual_departure,Actual_Arrival,TransitTime,seconds_late
timestamps,,,,,,,
2013-01-01 00:55:00,Bainbridge,Colman,55:00.0,05:56.0,37:37.0,32.0,656.0
2013-01-01 01:00:00,Bainbridge,Colman,NaN,NaN,NaN,NaN,NaN
2013-01-01 04:45:00,Bainbridge,NaN,45:00.0,NaN,NaN,NaN,NaN
2013-01-01 05:20:00,Bainbridge,Colman,20:00.0,19:01.0,49:40.0,30.0,-59.0
2013-01-01 06:20:00,Bainbridge,NaN,20:00.0,NaN,NaN,NaN,NaN


In [44]:
import pdb

In [26]:
d = datetime.datetime(2013, 1, 1, 1,0,)

In [27]:
d in df.index.to_series()

False

In [110]:
prev_date = df.iloc[0].name

tdelta = datetime.timedelta(minutes = 5)

stop = 0

#for i in range(1,(df.shape[0] - 1)):
while (not prev_date == datetime.datetime(2013, 12, 31, 23, 55)):
    if(df.iloc[i].name == (prev_date + tdelta)):
        prev_date = prev_date + tdelta
        continue
    
    prev_date = prev_date + tdelta
    example.name = prev_date
    df = df.append(example)
    
    #if (stop == 100):
    #    break
    #stop += 1

KeyboardInterrupt: 

In [107]:
df = df.sort_values(by = "timestamps")

In [108]:
df.to_csv("test.csv")

In [42]:
print(start_date)

2013-01-01 00:55:00
